<a href="https://colab.research.google.com/github/soraochi1626-cmd/economics-study/blob/main/econometrics/simple_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install py4macro
!pip install wooldridge

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 16.2 MB/s eta 0:00:00


In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import py4macro    # 属性を調べるsee()関数が含まれる
import statsmodels.formula.api as smf  # 回帰分析のパッケージ
import wooldridge  # wooldridgeのデータ・パッケージ

from scipy.stats import norm    # 正規分布関数
from numba import njit  # シミュレーションの計算を高速化する

# 警告メッセージを非表示
import warnings
warnings.filterwarnings("ignore")

#説明
説明変数が一つの単回帰(Simple Regression)を考える。<br>

$$y_i = \beta_0 + \beta_1x_i + u_i$$<br>
・$y_i$：非説明変数(従属変数)<br>
・$x_i$：説明変数(独立変数)<br>
・$u_i$：誤差項<br>
・$i = 1, 2, …, n$：観測値のインデックス<br>
・$\beta_0$：定数項(観測不可能なため推定する)<br>
・$\beta_1$：スロープ係数(観測不可能なため推定する
)




**最小二乗推定法**

$$ \sum_{i=1}^{n} (y_i - \beta_0 - \beta_1{x_i})^2 $$

を最小化する$\beta_0$と$\beta_1$を選択すると以下を得る：

$$\hat{\beta_0} = \bar{y} - \hat{\beta_1}\bar{x}$$
$$\hat{\beta_1} = \frac{Cov(x, y)}{Var(x)}$$

・^(ハット)がある変数($\hat{\beta_0}$と$\hat{\beta_1}$)を**OSL推定量**と呼ぶ<br>
・-(バー)がある変数($\bar{y}$)は平均値<br>
・$Cov(x, y)$は$x$と$y$の共分散<br>
・$Var(x)$は$x$の分散<br>
・$x$のように添字$i$を省いた場合は$x_1$から$x_n$までのすべての観測地という意味($y$も同じ)<br>

また以下を定義する

・予測値$\hat{y_i}$<br>$$\hat{y_i} = \hat{\beta_0} + \hat{\beta_1}{x_i}$$
・残差$\hat{u_i}$<br>$$\hat{u_i} = y_i - \hat{y_i}$$



###データ
Wooldridge (2019)で使われるデータセットは全てwooldridgeのパッケージに含まれている。そのパッケージからからデータを読み込んで使うことにする。

使う関数はdata()であり, 引数rなしで実行するとデータセットのリストが表示される。

In [4]:
wooldridge.data()

  J.M. Wooldridge (2019) Introductory Econometrics: A Modern Approach,
  Cengage Learning, 7th edition.

  401k       401ksubs    admnrev       affairs     airfare
  alcohol    apple       approval      athlet1     athlet2
  attend     audit       barium        beauty      benefits
  beveridge  big9salary  bwght         bwght2      campus
  card       catholic    cement        census2000  ceosal1
  ceosal2    charity     consump       corn        countymurders
  cps78_85   cps91       crime1        crime2      crime3
  crime4     discrim     driving       earns       econmath
  elem94_95  engin       expendshares  ezanders    ezunem
  fair       fertil1     fertil2       fertil3     fish
  fringe     gpa1        gpa2          gpa3        happiness
  hprice1    hprice2     hprice3       hseinv      htv
  infmrt     injury      intdef        intqrt      inven
  jtrain     jtrain2     jtrain3       jtrain98    kielmc
  labsup     lawsch85    loanapp       lowbrth     mathpnl
  meap00_01  

この中にあるwage1を使う。データセットを読み込むには、引数にデータセット名を文字列として入力する。返り値はDataFrameであり、回帰分析に使うためにdfに割り当てる。

In [5]:
df = wooldridge.data('wage1')

オプションdescription=Trueを追加するとデータの説明を表示することができる。

In [6]:
wooldridge.data('wage1',description=True)

name of dataset: wage1
no of variables: 24
no of observations: 526

+----------+---------------------------------+
| variable | label                           |
+----------+---------------------------------+
| wage     | average hourly earnings         |
| educ     | years of education              |
| exper    | years potential experience      |
| tenure   | years with current employer     |
| nonwhite | =1 if nonwhite                  |
| female   | =1 if female                    |
| married  | =1 if married                   |
| numdep   | number of dependents            |
| smsa     | =1 if live in SMSA              |
| northcen | =1 if live in north central U.S |
| south    | =1 if live in southern region   |
| west     | =1 if live in western region    |
| construc | =1 if work in construc. indus.  |
| ndurman  | =1 if in nondur. manuf. indus.  |
| trcommpu | =1 if in trans, commun, pub ut  |
| trade    | =1 if in wholesale or retail    |
| services | =1 if in services indus.  

このデータの使い次の問題を考える。

###教育水準は賃金にどのような影響を与えるか

・非説明変数：wage(平均時給, 単位：米ドル)
・説明変数：educ(教育年数, 単位：年)

まず関係する変数だけを取り出してみる。

In [7]:
df = df.loc[:,['wage','educ']]
df.head(3)

,wage,educ
0,3.10,11
1,3.24,12
2,3.00,11


DataFrameの特徴を確認する。

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 526 entries, 0 to 525
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   wage    526 non-null    float64
 1   educ    526 non-null    int64  
dtypes: float64(1), int64(1)
memory usage: 8.3 KB


記述統計も確認する。

In [9]:
df.describe()

,wage,educ
count,526.000000,526.000000
mean,5.896103,12.562738
std,3.693086,2.769022
min,0.530000,0.000000
25%,3.330000,12.000000
50%,4.650000,12.000000
75%,6.880000,14.000000
max,24.980000,18.000000


#手計算

In [13]:
mat_wage_educ = np.cov(df['wage'], df['educ'])
mat_wage_educ

array([[13.63888436,  4.150864  ],
       [ 4.150864  ,  7.66748506]])

共分散の値をcov_wage_educに割り当てる。

In [14]:
cov_wage_educ = mat_wage_educ[0,1]
cov_wage_educ

np.float64(4.150863998572981)

wageとeducの分散は次のコードで取得できる。

In [15]:
mat_wage_educ[0,0], mat_wage_educ[1,1]

(np.float64(13.638884363775006), np.float64(7.667485062466053))

次のように計算しても同じ値となる。

In [16]:
var_wage = df['wage'].var(ddof=1)
var_educ = df['educ'].var(ddof=1)
var_wage, var_educ

(13.638884363775011, 7.667485062466044)

それぞれの変数の平均を計算する。

In [17]:
mean_wage = df['wage'].mean()
mean_educ = df['educ'].mean()

上の式に従って$\beta_1$の推定値($\hat{\beta_1}$)も計算する。

In [18]:
b1hat = cov_wage_educ / var_educ
b1hat

np.float64(0.5413592546651751)

**＜結果の解釈＞**

b1hatは次式の推定結果である。
$$y = \beta_0 + \beta_1{x} + u$$
この式を$x$について微分すると
$$\frac{dy}{dx} = \frac{yの変化}{xの増加} = (xが1単位増加した場合のyの変化) = \beta_1$$
となる。即ち、$\beta_1$は教育年数が１年増加した場合, 平均時給は何ドル変化するかを示している。OLS推定結果は教育年数が１円増えると賃金は約0.54ドル上昇することを示している。

同様に$\beta_0$の推定値($\hat{\beta_0}$)も計算する。


In [20]:
b0hat = mean_wage - b1hat * mean_educ
b0hat

np.float64(-0.9048516119572181)

#statmodelsを使う
Pythonでは簡単なコードで回帰分析に関する多くの計算を行うことができるパッケージが提供されている。その内, 代表的なパッケージがstatsmodelsである。次の順番で使い方を説明する。<br>
・olsサブパッケージを読み込む<br>
・回帰式を定義する<br>
・推定式を定義する<br>
**step 1:サブパッケージの読み込み**<br>
statsmodelsには多くのサブパッケージや関数があるが, ここでは回帰式の文字列で書くことができるols関数を使う。(他のコードの書き方もあるが, 他のトピックで扱う。) oldはこのページの冒頭で次のコードでインポートされている。<br>




In [21]:
from statsmodels.formula.api import ols

**step 2:回帰式の定義**<br>
・式を' 'で囲む(文字列)<br>
・定数項である($\beta_0$)は自動的に挿入される。<br>


定数項がある場合：被説明変数 ~ 説明変数<br>
定数項がない場合：被説明変数 ~ -1 + 説明変数

wageを被説明変数にし, 定数項を加えてeducを説明変数とする場合, 次のように書くことになる。